- 필요한 패키지 임포트

In [64]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import FinanceDataReader as fdr
import io 
import zipfile
import xml.etree.ElementTree as et 
import json
import requests

- api 넘버 가져오기

In [65]:
api_num = pd.read_csv('/Users/kimhansaem/workspace/11.DataAnalysis/01.데이터분석/data/apinumber.txt', sep='\s+', header=None,
                                names=['column_name'])
api_number = api_num['column_name']

- 상장 기업 전체 기업 이름, 기업코드 api 이용하여 불러오기 

In [66]:
def get_corpcode(crtfc_key): 
    params = {'crtfc_key':crtfc_key}
    url = "https://opendart.fss.or.kr/api/corpCode.xml" 
    items_en = ["corp_code", "corp_name", "stock_code", "modify_date"] 
    
    ## Column명을 한글로 하려면 아래부분 쌍따옴표안을 수정하면 됨 
    items_kr = ["corp_code", "corp_name", "stock_code", "modify_date"] 
    results = requests.get(url,params=params) 
    uzfile = zipfile.ZipFile(io.BytesIO(results.content))
    final = uzfile.open(uzfile.namelist()[0]) 
    root = et.fromstring(final.read().decode('utf-8')) 
    data = [] 
    for child in root: 
        if len(child.find('stock_code').text.strip()) > 1: 
            data.append([]) 
            for item in items_en: 
                data[-1].append(child.find(item).text) 
    df = pd.DataFrame(data, columns = items_kr) 
    ##print(df) 
    return df

- 다트에 있는 기업 리스트 추출

In [67]:
crtfc_key = api_number[0]
code_df = get_corpcode(crtfc_key)
code_df.head()

,corp_code,corp_name,stock_code,modify_date
0,00260985,한빛네트,036720,20170630
1,00264529,엔플렉스,040130,20170630
2,00358545,동서정보기술,055000,20170630
3,00231567,애드모바일,032600,20170630
4,00247939,씨모스,037600,20170630


- 다트에 재무제표 올라와 있는 기업 개수: 3418

In [68]:
code_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3422 entries, 0 to 3421
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   corp_code    3422 non-null   object
 1   corp_name    3422 non-null   object
 2   stock_code   3422 non-null   object
 3   modify_date  3422 non-null   object
dtypes: object(4)
memory usage: 107.1+ KB


- 반도체기업 이름, stock code 추출

In [69]:
# 반도체 기업 이름 추출
company_name_df = pd.read_csv('/Users/kimhansaem/workspace/11.DataAnalysis/01.데이터분석/data/반도체 기업 목록.txt', sep='\s+', header=None,
                                names=['column_index', 'column_name'])

In [70]:
# 반도체 기업 추출 확인
company_names = company_name_df['column_index'].values.tolist()
print(company_names[:5])

['큐에스아이', '예스티', '미코', '신성이엔지', '코세스']


- 반도체기업 주식코드 추출

In [73]:
df_corp = code_df[code_df['corp_name'].isin(company_names)]
df_corp.head(3)

,corp_code,corp_name,stock_code,modify_date
611,00686068,신성이엔지,104110,20171208
775,00625942,아이원스,114810,20220117
808,00484682,엘오티베큠,083310,20211123


- 주식코드 안 맞는 회사들 검색

In [74]:
df_coodeee = df_corp.copy()
df_coodeee

,corp_code,corp_name,stock_code,modify_date
611,00686068,신성이엔지,104110,20171208
775,00625942,아이원스,114810,20220117
808,00484682,엘오티베큠,083310,20211123
809,00492894,젬백스,082270,20211124
831,00479787,인텍플러스,064290,20211129
...,...,...,...,...
3320,00223434,에프에스티,036810,20220310
3362,00535676,테크윙,089030,20220314
3363,00659161,싸이맥스,160980,20220314
3408,00121686,엠케이전자,033160,20220228


In [75]:
df_coodeee['modify_date'] = df_coodeee['modify_date'].apply(lambda x: int(x.replace(',','')))
df_coodeee.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 127 entries, 611 to 3413
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   corp_code    127 non-null    object
 1   corp_name    127 non-null    object
 2   stock_code   127 non-null    object
 3   modify_date  127 non-null    int64 
dtypes: int64(1), object(3)
memory usage: 5.0+ KB


- 신성이엔지만 인베스팅닷컴에 있는 주식코드와 상이함

In [48]:
df_coodeee = df_coodeee[(df_coodeee['modify_date'] <= 20220101)]
df_coodeee

,corp_code,corp_name,stock_code,modify_date
611,00686068,신성이엔지,011930,20171208
808,00484682,엘오티베큠,083310,20211123
809,00492894,젬백스,082270,20211124
831,00479787,인텍플러스,064290,20211129
836,00158219,시그네틱스,033170,20211229
841,00372226,티에스이,131290,20211201
881,00325112,프로텍,053610,20211209
976,00377018,기가레인,049080,20211201
999,00530413,코디엠,224060,20211210
1004,00563545,테스나,131970,20211210


- 신성이엔지 주식코드 수정

In [95]:
ss_mask = df_corp["corp_name"] =="신성이엔지"
df_corp.loc[ss_mask, "stock_code"] = "011930"
df_corp.head()

,corp_code,corp_name,stock_code,modify_date
611,00686068,신성이엔지,011930,20171208
775,00625942,아이원스,114810,20220117
808,00484682,엘오티베큠,083310,20211123
809,00492894,젬백스,082270,20211124
831,00479787,인텍플러스,064290,20211129


- 반도체기업 주식코드 수정 완료

In [96]:
df_corp2 = df_corp.drop(['modify_date'], axis=1)
df_corp2

,corp_code,corp_name,stock_code
611,00686068,신성이엔지,011930
775,00625942,아이원스,114810
808,00484682,엘오티베큠,083310
809,00492894,젬백스,082270
831,00479787,인텍플러스,064290
...,...,...,...
3320,00223434,에프에스티,036810
3362,00535676,테크윙,089030
3363,00659161,싸이맥스,160980
3408,00121686,엠케이전자,033160


In [97]:
df_corp2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 127 entries, 611 to 3413
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   corp_code   127 non-null    object
 1   corp_name   127 non-null    object
 2   stock_code  127 non-null    object
dtypes: object(3)
memory usage: 4.0+ KB


- 반도체기업 stock_code 리스트로 가져오기

In [125]:
company_codes = df_corp2['stock_code'].values.tolist()
company_codes

['011930',
 '114810',
 '083310',
 '082270',
 '064290',
 '033170',
 '131290',
 '053610',
 '402340',
 '049080',
 '224060',
 '131970',
 '122640',
 '036540',
 '061970',
 '087600',
 '092070',
 '077500',
 '039030',
 '031980',
 '217190',
 '050090',
 '098120',
 '064520',
 '093640',
 '080520',
 '067310',
 '059120',
 '217500',
 '036930',
 '222160',
 '200710',
 '330860',
 '089790',
 '240810',
 '303030',
 '029460',
 '036200',
 '073570',
 '200470',
 '059090',
 '030530',
 '119830',
 '045300',
 '306620',
 '032580',
 '005290',
 '025560',
 '003160',
 '077360',
 '005930',
 '101160',
 '064760',
 '102120',
 '171010',
 '033640',
 '042700',
 '089530',
 '011930',
 '117670',
 '101400',
 '183300',
 '084370',
 '123010',
 '357780',
 '000990',
 '092600',
 '195870',
 '058470',
 '208860',
 '095340',
 '080220',
 '253590',
 '096610',
 '147760',
 '101490',
 '080580',
 '097800',
 '311320',
 '052860',
 '074600',
 '142210',
 '319660',
 '089890',
 '052900',
 '348210',
 '269620',
 '006200',
 '267320',
 '095610',
 '356860',

- KRX 상장사 전체 불러오기

In [99]:
df = fdr.StockListing('KRX')
df.head()

,Symbol,Market,Name,Sector,Industry,ListingDate,SettleMonth,Representative,HomePage,Region
0,060310,KOSDAQ,3S,전자부품 제조업,반도체 웨이퍼 캐리어,2002-04-23,03월,김세완,http://www.3sref.com,서울특별시
1,095570,KOSPI,AJ네트웍스,산업용 기계 및 장비 임대업,"렌탈(파렛트, OA장비, 건설장비)",2015-08-21,12월,박대현,http://www.ajnet.co.kr,서울특별시
2,006840,KOSPI,AK홀딩스,기타 금융업,지주사업,1999-08-11,12월,"채형석, 이석주(각자 대표이사)",http://www.aekyunggroup.co.kr,서울특별시
3,054620,KOSDAQ,APS홀딩스,기타 금융업,인터넷 트래픽 솔루션,2001-12-04,12월,정기로,http://www.apsholdings.co.kr,경기도
4,265520,KOSDAQ,AP시스템,특수 목적용 기계 제조업,디스플레이 제조 장비,2017-04-07,12월,김영주,http://www.apsystems.co.kr,경기도


In [100]:
# df9 = df.rename(columns={'Symbol':'stock_code'})
# df9

,stock_code,Market,Name,Sector,Industry,ListingDate,SettleMonth,Representative,HomePage,Region
0,060310,KOSDAQ,3S,전자부품 제조업,반도체 웨이퍼 캐리어,2002-04-23,03월,김세완,http://www.3sref.com,서울특별시
1,095570,KOSPI,AJ네트웍스,산업용 기계 및 장비 임대업,"렌탈(파렛트, OA장비, 건설장비)",2015-08-21,12월,박대현,http://www.ajnet.co.kr,서울특별시
2,006840,KOSPI,AK홀딩스,기타 금융업,지주사업,1999-08-11,12월,"채형석, 이석주(각자 대표이사)",http://www.aekyunggroup.co.kr,서울특별시
3,054620,KOSDAQ,APS홀딩스,기타 금융업,인터넷 트래픽 솔루션,2001-12-04,12월,정기로,http://www.apsholdings.co.kr,경기도
4,265520,KOSDAQ,AP시스템,특수 목적용 기계 제조업,디스플레이 제조 장비,2017-04-07,12월,김영주,http://www.apsystems.co.kr,경기도
...,...,...,...,...,...,...,...,...,...,...
7675,000545,KOSPI,흥국화재우,NaN,NaN,NaT,NaN,NaN,NaN,NaN
7676,003280,KOSPI,흥아해운,해상 운송업,외항화물운송업(케미컬탱커),1976-06-29,12월,이환구,http://www.heung-a.com,서울특별시
7677,037440,KOSDAQ,희림,"건축기술, 엔지니어링 및 관련 기술 서비스업",설계 및 감리용역,2000-02-03,12월,"정영균, 이목운, 허철호 (각자대표)",http://www.heerim.com,서울특별시
7678,419890,KOSPI,히어로즈 단기채권ESG액티브,NaN,NaN,NaT,NaN,NaN,NaN,NaN


In [101]:
# df9 = df9[['stock_code', 'Name']]
# df9.head(3)

,stock_code,Name
0,060310,3S
1,095570,AJ네트웍스
2,006840,AK홀딩스


In [102]:
# df9.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7680 entries, 0 to 7679
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   stock_code  7680 non-null   object
 1   Name        7680 non-null   object
dtypes: object(2)
memory usage: 180.0+ KB


In [103]:
# df2 = df9[df9['stock_code'].isin(company_codes)]
# df2.head(5)

,stock_code,Name
82,000990,DB하이텍
121,083450,GST
181,095340,ISC
1201,092220,KEC
1270,052900,KMH하이텍


In [105]:
# abc = pd.merge(df_corp2, df2, on='stock_code', how='left')
# abc

,corp_code,corp_name,stock_code,Name
0,00686068,신성이엔지,011930,신성이엔지
1,00625942,아이원스,114810,아이원스
2,00484682,엘오티베큠,083310,엘오티베큠
3,00492894,젬백스,082270,젬백스
4,00479787,인텍플러스,064290,인텍플러스
...,...,...,...,...
122,00223434,에프에스티,036810,에프에스티
123,00535676,테크윙,089030,테크윙
124,00659161,싸이맥스,160980,싸이맥스
125,00121686,엠케이전자,033160,엠케이전자


In [110]:
abc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 127 entries, 0 to 126
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   corp_code   127 non-null    object
 1   corp_name   127 non-null    object
 2   stock_code  127 non-null    object
 3   Name        127 non-null    object
dtypes: object(4)
memory usage: 5.0+ KB


In [108]:
# abc.isnull().sum()

corp_code     0
corp_name     0
stock_code    0
Name          0
dtype: int64

- 한 기업 빠지는데 뭔지 모르겠음

In [111]:
df2 = df9[df9['stock_code'].isin(company_codes)]
df2.head(5)

,stock_code,Name
82,000990,DB하이텍
121,083450,GST
181,095340,ISC
1201,092220,KEC
1270,052900,KMH하이텍


In [112]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 126 entries, 82 to 7649
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   stock_code  126 non-null    object
 1   Name        126 non-null    object
dtypes: object(2)
memory usage: 3.0+ KB


- 주가정보 수집

In [115]:
name = 'Open	High	Low	Close	Volume	Change'
tag = name.split()
tag

['Open', 'High', 'Low', 'Close', 'Volume', 'Change']

In [122]:
item_list = []
for item_code in company_codes:
    stockpriceinfo = fdr.DataReader(item_code, "2020", "2021")[tag]
    item_list.append(stockpriceinfo)

In [126]:
company_codes2 = df_corp2['stock_code'][:1].values.tolist()
company_codes2

['011930']

- 이 코드로 5년치 전체 조회 가능

In [134]:
item_list = []
for item_code in company_codes2:
    stockpriceinfo = fdr.DataReader(item_code, "2017", "2022")[tag]
    item_list.append(stockpriceinfo)

In [135]:
df_onecorp = pd.concat(item_list, axis=0)
df_onecorp.columns = tag
df_onecorp.head()

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2017-01-02,2160,2180,2110,2140,1796884,-0.031674
2017-01-03,2120,2150,2100,2100,1359835,-0.018692
2017-01-04,2095,2150,2075,2120,954309,0.009524
2017-01-05,2150,2150,2080,2110,1115377,-0.004717
2017-01-06,2115,2115,2060,2075,1306203,-0.016588


In [137]:
# 5년치 다 나오는지 확인
df_onecorp.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1229 entries, 2017-01-02 to 2021-12-30
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Open    1229 non-null   int64  
 1   High    1229 non-null   int64  
 2   Low     1229 non-null   int64  
 3   Close   1229 non-null   int64  
 4   Volume  1229 non-null   int64  
 5   Change  1229 non-null   float64
dtypes: float64(1), int64(5)
memory usage: 67.2 KB


In [ ]:
item_list = []
for item_code in company_codes:
    stockpriceinfo2 = fdr.DataReader(item_code, "2020", "2021")[tag]
    item_list.append(stockpriceinfo2)

In [ ]:
# Base : 기본적으로 테이블을 다루기 위한 여러 내용들이 들어있음.
# - 대표적으로 CRUD에 대한 메소드가 들어있다.
Base = declarative_base() # Base 클래스를 얻어내는 함수

# Base 클래스 상속
# 데이터만 담겨있는 클래스(객체) : Value Object VO
class Price(Base):
    
    __tablename__ = "price_list" # 테이블 이름
    
    # 컬럼 정의
    #  - 컬럼명, 컬럼 데이터 타입
    Date = Column(Datetime, primary_key=True)
    Open = Column(Integer) 
    High = Column(Integer)
    low = Column(Integer)
    Volume = Column(Integer)
    Change = Column(Integer)
    
    # 생성자에서는 각 컬럼에 데이터를 매핑
    def __init__(self, Date, Open, High, Low, Volume, Change):
        # self.변수는 컬럼명과 반드시 일치
        self.Date = Date
        self.Open = Open
        self.High = High
        self.Low = Low
        self.Volume = Volume
        self.Change = Change
    
    # repr 함수
    def __repr__(self):
        return "<Price {}, {}, {}, {}, {}, {}>".format(self.Date, self.Open, self.High, self.Low, self.Volume, self.Change)
    

In [131]:
df = fdr.DataReader("011930", '2022')
df.head()

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2022-01-03,2165,2185,2125,2165,1022712,0.009324
2022-01-04,2160,2180,2140,2175,970401,0.004619
2022-01-05,2165,2175,2085,2105,1351951,-0.032184
2022-01-06,2050,2080,2050,2050,1084620,-0.026128
2022-01-07,2040,2070,2040,2055,750944,0.002439


In [4]:
eur = fdr.DataReader("EUR/USD","2022")

In [ ]:
df = fdr.DataReader("005930", '2019')

In [55]:
!pip install sqlalchemy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 20.7 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for sqlalchemy: filename=SQLAlchemy-1.4.32-cp38-cp38-macosx_11_0_arm64.whl size=1536908 sha256=05412114bc399b2f8f831d9a1216eb03f0a40ff499740bd0a56341c7155bc7b3
  Stored in directory: /Users/kimhansaem/Library/Caches/pip/wheels/f1/89/a6/de94839bd1eae1e97da3e026f85938bc8afca6739bda1df206
Successfully built sqlalchemy
You should consider upgrading via the '/Users/kimhansaem/miniconda3/bin/python -m pip install --upgrade pip' command.


In [56]:
import MySQLdb
from sqlalchemy import create_engine

In [58]:
engine = create_engine("mysql+mysqldb://saem:"+"0000"+"@ip주소/financedb", encoding='utf-8')
conn = engine.connect()

In [62]:
# MySQL에 저장하기
# pandas의 to_sql() 함수 사용 저장
# DB하이텍 연습
df.to_sql(name='price_list', con=engine, if_exists='append')

In [ ]:
for data in dates:
    data.age = 123123
    sess.add(data)
sess.commit()

In [ ]:
sql = """
CREATE TABLE `financedb`.`companycodes.index[i]` (
  Date DATETIME primary key,
  Open VARCHAR(45) NOT NULL,
  High VARCHAR(45) NOT NULL,
  Low VARCHAR(45) NOT NULL,
  Close VARCHAR(45) NOT NULL,
  Change VARCHAR(45) NOT NULL)
  """


# many insert
users = [
    User("mino12321", "asaas@daum.net", 22, "2021-11-30"),
    User("cofff123123", "aasdasd@naver.com", 30, "2020-11-11")
]

Session = sessionmaker(engine)
sess = Session()
sess.add_all(datas)
sess.commit()

In [ ]:

db = MySQLdb.connect(
    "--", # database server public ip
    "saem",        # user
    "0000",        # password
    "financedb"        # database
)

db

In [ ]:
QUERY ="""
    SELECT *
    FROM city
"""

df_city = pd.read_sql(QUERY, engine)
df_city.head()

In [ ]:
cursor.execute(QUERY)
result = cursor.fetchall()
result